In [ ]:
import preprocessing as pp
from claim_verifier import ClaimCheck
from contrarian_claims import contrarian_claims_full
import os
# import nltk
# from nltk.tokenize import sent_tokenize
from frame_extractor import FrameExtract
from tqdm import tqdm
import pickle

TEST A SINGLE ARTICLE

In [ ]:
newspaper = "telegraph"
verifier = ClaimCheck()
#%%
# find all articles

filename = "'Greens hindering the global warming fight' Exaggerated claims mislead public, but world still needs.txt"
text = pp.load_text(f"articles/{newspaper}/txt/{filename}")
date, length, author, title = pp.extract_metadata(text, filename)
# Loop through all files in the input folder
# filename_list = [filename for filename in os.listdir(input_folder) if filename.endswith(".txt")]
# df_path = f"{newspaper}_claims.csv"
# dataframe = pp.load_or_create_df(df_path)

In [ ]:
# get sentences
#sentences = sent_tokenize(text)[1:-1]

In [ ]:
# Is this article about climate change?
question = "Does this TEXT discuss climate change?"
answer = verifier.claim_check(text = text, question = question)
print(answer==True)

In [ ]:
text_claims = {'claims': [], 'claim_ids': []}

for claim_id, claim_details in tqdm(contrarian_claims_full.items()):
    claim_description = claim_details["description"]
    question = f"Does the TEXT explicitly or implicitly present, discuss, or relate to the following claim: {claim_description}?"
    match = verifier.claim_check(text = text, question = question)
    if match:
        text_claims['claims'].append(claim_description)
        text_claims['claim_ids'].append(claim_id)

In [ ]:
# f = open("claim_test.pkl", 'wb')
# pickle.dump(text_claims, f)
# f.close()

#text_claims = pickle.load(open("claim_test.pkl", 'rb'))

In [ ]:
FX = FrameExtract(claims_file=contrarian_claims_full)

In [ ]:
all_frames = {id: {'best_frame': '', 'candidate_frames': []}}
for k in range(len(text_claims['claim_ids'])):
    if k > 0:
        break
    claim_description = text_claims['claims'][k]
    id = text_claims['claim_ids'][k]

    detective_thoughts = []
    # generate agent frame detection thought
    for agent in range(4):
        thought = FX.CoT_detect_framing(article = text, claim = claim_description)
        print(thought)
        detective_thoughts.append(thought)

In [ ]:
detective_thoughts

In [ ]:
is_framing = FX.hivemind_detection(article = text, claim = claim_description, thought_list=detective_thoughts)


In [ ]:
frame_explanations = []
# generate agent frame explanation thought
for agent in range(4):
    thought = FX.CoT_explain_framing(article = text, claim = claim_description)
    print(thought)
    frame_explanations.append(thought)

final_explanation = FX.update_explanation(article = text, claim = claim_description, thought_list=frame_explanations)



In [ ]:
print(final_explanation)

In [ ]:
candidate_summaries = []
for agent in range(4):
    thought = FX.generate_frame_summary(article = text, claim = claim_description, explanation=final_explanation)
    print(thought)
    candidate_summaries.append(thought)


In [ ]:
text_claims

In [ ]:
final_frame = FX.hivemind_choose_best_frame(article = text, claim = claim_description, thought_list = candidate_summaries)
print(final_frame)
all_frames[id]['best_frame'] = final_frame
all_frames[id]['candidate_frames'] = candidate_summaries